In [1]:
import numpy as np
import xarray as xr
import copy 

In [ ]:
#Use CDO for regridding data to MPI-GE grid
#cdo -remapbil,MPI_GE_grid.txt in_file_original.nc out_file_GEgrid.nc

In [2]:
stepsize = 1
window = 15
start = int((window-1)/2)
ref_min = 1985
ref_max = 2014

In [3]:
# compute climatology
path = '/work/uo1075/u241308/reanalysis/eobs/eobs_hw_projections/'
file = 'tasmax_1950_2022_GEgrid2.nc'

with xr.open_dataset(path+file) as data1:
    data1 = data1.tx
    #Get rid off 29th of februarys
    data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))

#flatten lon/lat
data_flat = data1.values.reshape(data1.shape[0],-1) 

#calculate the climatology for the reference time
climatology = []
r_steps = 365-window+1 #number of different running windows
for r in range(r_steps):
    sel_days = []
    for y in range(ref_min,ref_max+1):
        data_sel = data_flat[(data1.time.dt.year==y)]
        data_sel = data_sel[r:r+window]
        sel_days.append(data_sel)
    
    sel_days_mean = np.array(sel_days).mean(axis=(0,1))
    climatology.append(sel_days_mean)
climatology = np.array(climatology)

climatology_complete = np.zeros((365,climatology.shape[1])) #fill values missing due to running window
start = int((window-1)/2)
climatology_complete[:start] = climatology[0]
climatology_complete[-start:] = climatology[-1]
climatology_complete[start:-start] = climatology

In [4]:
#reshape lon/lat to original format
climatology_expanded = climatology_complete.reshape(-1,data1.shape[1],data1.shape[2])
    
#calculate anomalies
for y in np.unique(data1.time.dt.year):
    year_sel = data1[data1.time.dt.year==y].values
    anomalies1 = year_sel - climatology_expanded
    data1[data1.time.dt.year==y] = anomalies1

# ------------------ save data -----------------------
path = '/work/uo1075/u241308/reanalysis/eobs/eobs_hw_projections/'
file = 'tasmax_1950_2022_GEgrid2_anomaly.nc'
data1.to_netcdf(path+file)